In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns

In [2]:
pd.options.display.float_format = '{:.2f}'.format

In [3]:
dates_to_visualize = [
                                                                                            "2018-09", "2018-10", "2018-11", "2018-12",
    "2019-01", "2019-02", "2019-03", "2019-04", "2019-05", "2019-06", "2019-07", "2019-08", "2019-09", "2019-10", "2019-11", "2019-12",
    "2020-01", "2020-02", "2020-03", "2020-04", "2020-05", "2020-06", "2020-07", "2020-08", "2020-09", "2020-10", "2020-11", "2020-12",
    "2021-01", "2021-02", "2021-03", "2021-04", "2021-05", "2021-06", "2021-07", "2021-08", "2021-09", "2021-10", "2021-11", "2021-12",
    "2022-01", "2022-02", "2022-03", "2022-04", "2022-05", "2022-06", "2022-07", "2022-08", "2022-09", "2022-10", "2022-11", "2022-12",
    "2023-01", "2023-02", "2023-03", "2023-04", "2023-05", "2023-06", "2023-07", "2023-08", "2023-09", "2023-10", "2023-11", "2023-12",
    "2024-01", "2024-02", "2024-03", "2024-04", 
]

In [4]:
available_data_files = list(Path("../data/httparchive_metrics/nel_collector_provider_usage").glob("*.parquet"))

used_data_files = list(filter(lambda file: file.stem in dates_to_visualize, available_data_files))
used_data_files

### Aggregate result to visualize


In [5]:
from results.result_utils import date_to_text_format, concat_data_from_files, MONTHS_MAP

result = concat_data_from_files(used_data_files)

result['date_formatted'] = result['date'].map(date_to_text_format)

result['as_primary'] = result['as_primary'].astype('UInt32')
result['as_secondary'] = result['as_secondary'].astype('UInt32')
result['among_fallback'] = result['among_fallback'].astype('UInt32')

result

### Top 5 Primary Collector Providers per month


In [6]:
top_5_data = result[result.index < 5][['date', 'providers', 'as_primary', 'share_as_primary']]

count_by_date = result.groupby(['date'])['as_primary'].agg(lambda group: len([x for x in group if x > 0]))
top_5_data['count'] = top_5_data['date'].map(count_by_date)

top_5_data = top_5_data.reindex(columns=['date', 'count', 'providers', 'as_primary', 'share_as_primary'])
top_5_data['date'] = top_5_data['date'].map(date_to_text_format)
# top_5_data.groupby(['date', 'count', 'providers']).first()
top_5_data.sort_values(['date', 'share_as_primary'], ascending=[True, False])

### Top 5 Secondary Collector Providers per month


In [7]:
secondaries = result[['date', 'providers', 'as_secondary', 'share_as_secondary']]
secondaries = secondaries[secondaries['as_secondary'] > 0]

top_5_data = pd.DataFrame()
for month in dates_to_visualize:
    monthly_data = secondaries[secondaries['date'] == month]
    
    monthly_data.reset_index(drop=True, inplace=True)
    top_5_data = pd.concat([top_5_data, monthly_data])

top_5_data = top_5_data[top_5_data.index < 5]

count_by_date = result.groupby(['date'])['as_secondary'].agg(lambda group: len([x for x in group if x > 0]))
top_5_data['count'] = top_5_data['date'].map(count_by_date)

top_5_data = top_5_data.reindex(columns=['date', 'count', 'providers', 'as_secondary', 'share_as_secondary'])
top_5_data['date'] = top_5_data['date'].map(date_to_text_format)
# # top_5_data.groupby(['date', 'count', 'providers']).first()
top_5_data.sort_values(['date', 'share_as_secondary'], ascending=[True, False]).reset_index(drop=True)

### Top 5 Collector Providers by occurrence

In [8]:
occurrence_data = result[['date', 'providers', 'as_primary', 'as_secondary', 'among_fallback']]

all_occurrences = pd.DataFrame()
for month in dates_to_visualize:
    monthly_data = occurrence_data[occurrence_data['date'] == month].copy()
    
    monthly_data['total_occurrences'] = monthly_data['as_primary'] + monthly_data['as_secondary'] + monthly_data['among_fallback']
    monthly_data.drop(columns=['as_primary', 'as_secondary', 'among_fallback'], inplace=True)
    
    monthly_data['date'] = monthly_data['date'].map(date_to_text_format)
    
    monthly_data.reset_index(drop=True, inplace=True)
    
    all_occurrences = pd.concat([all_occurrences, monthly_data])
    
top_5_data = all_occurrences[all_occurrences.index < 5]
# top_5_data = all_occurrences
top_5_data

#### The TOP provider over the analyzed time

In [54]:
to_plot = all_occurrences.copy()
to_plot = to_plot[to_plot.index == 0]

to_plot['date_fmt'] = to_plot['date'].map(lambda date: pd.to_datetime(date))
to_plot['providers'] = to_plot['providers'].astype('category')

sns.set_style('whitegrid')
g = sns.catplot(to_plot, x='date_fmt', y='providers', hue='providers', kind="swarm", height=4, aspect=2.5)
# g.fig.suptitle('Najvyužívanješí poskytovatelia NEL kolektorov za skúmané obdobie', fontweight='bold')
ax = g.axes[0][0]

# sns.barplot(to_plot, x='date_fmt', y='total_occurrences', ax=ax, hue='providers')

ax.xaxis.remove_overlapping_locs = False

years = mdates.YearLocator()
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(mdates.DateFormatter("\n\n%Y"))

months = mdates.MonthLocator(interval=1)
months_fmt = mdates.DateFormatter('%b')
ax.xaxis.set_minor_locator(months)
ax.xaxis.set_minor_formatter(months_fmt)

ax.tick_params(axis='x', which="minor", rotation=90, labelsize=9)

plt.grid(True, which='minor', linestyle='--',)

max_employment = to_plot.groupby(['providers'], observed=True).agg(max=('total_occurrences', 'max')).values
ax.legend(list(map(lambda val: '{:,.0f}'.format(val[0]), max_employment)), title='Maximum obsluhovaných domén', loc="lower right")

ax.set(xlabel='Dátum', ylabel='Poskytovateľ NEL kolektorov')

plt.tight_layout()
plt.savefig('graphs/httparchive_nel_collector_provider_top_1_over_time.pdf')

#### GRAPH - Number of collector providers over the time

In [20]:
to_plot = all_occurrences.copy()
to_plot = to_plot.groupby(['date'], as_index=False).agg(pcount=('providers', 'count'))


to_plot['date_fmt'] = to_plot['date'].map(lambda date: pd.to_datetime(date))
to_plot.drop(columns=['date'], inplace=True)
to_plot = to_plot.sort_values(by=['date_fmt'])
to_plot.reset_index(inplace=True)
to_plot = to_plot.reindex(columns=['date_fmt', 'pcount']) 

g = sns.relplot(to_plot, x='date_fmt', y='pcount', kind='line', color='orange', height=5, aspect=2.1)
ax = g.axes[0][0]

ax.yaxis.get_major_formatter().set_scientific(False)
ax.yaxis.get_major_formatter().set_useOffset(False)

ax.xaxis.remove_overlapping_locs = False
years = mdates.YearLocator()
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(mdates.DateFormatter("\n\n%Y"))

months = mdates.MonthLocator(interval=1)
months_fmt = mdates.DateFormatter('%b')
ax.xaxis.set_minor_locator(months)
ax.xaxis.set_minor_formatter(months_fmt)

ax.tick_params(axis='x', which="minor", rotation=90, labelsize=10)


ax.set(xlabel='Dátum', ylabel='Počet poskytovateľov NEL kolektorov')

plt.grid(True, which='minor', linestyle='--',)

ax.yaxis.set_major_locator(ticker.FixedLocator(np.arange(0, 340, 20)))
ax.tick_params(labelleft=False, labelright=True)

ax.lines[0].set_marker('.')
ax.lines[0].set_markersize(10)
ax.lines[0].set_markeredgecolor('gray')
ax.lines[0].set_markeredgewidth(1.5)

plt.tight_layout()
plt.savefig('graphs/httparchive_nel_collector_provider_count.pdf')


#### TABLE - Stats for the latest collector providers

In [11]:
all_data = all_occurrences.copy()
all_data['date_fmt'] = all_data['date'].map(lambda date: pd.to_datetime(date))
all_data.drop(columns=['date'], inplace=True)
all_data = all_data.sort_values(by=['date_fmt'])
all_data.reset_index(inplace=True)
all_data = all_data.reindex(columns=['date_fmt', 'providers', 'total_occurrences'])


latest_data = all_data[all_data['date_fmt'] == all_data['date_fmt'].max()].copy()
latest_data['total_employment'] = latest_data['total_occurrences'].sum()
latest_data['provider_share'] = latest_data['total_occurrences'] / latest_data['total_employment'] * 100

latest_data.sort_values(by=['total_occurrences'], ascending=False, inplace=True)
latest_c_p_stats = latest_data.head(10).copy()
latest_c_p_stats.reset_index(inplace=True, drop=True)


first_occurrences_by_provider = result[result['providers'].isin(latest_c_p_stats['providers'])].groupby('providers')['date'].min()
latest_c_p_stats['first_occurrence'] = latest_c_p_stats['providers'].map(first_occurrences_by_provider).map(date_to_text_format)


first_occurrence_employment_by_provider = all_occurrences[all_occurrences['providers'].isin(latest_c_p_stats['providers'])].groupby('providers').agg({'date': 'min', 'total_occurrences': 'first'})['total_occurrences']
latest_c_p_stats['first_occurrence_employment'] = latest_c_p_stats['providers'].map(first_occurrence_employment_by_provider)

latest_c_p_stats.rename(columns={'total_occurrences': 'current_employment', 'total_employment': 'current_total_employment', 'provider_share': 'current_share'}, inplace=True)
latest_c_p_stats = latest_c_p_stats[['date_fmt', 'providers', 'first_occurrence', 'first_occurrence_employment', 'current_employment', 'current_total_employment', 'current_share']]


# IMPORTANT: Next cells will use the current 'latest_c_p_stats' as input table!!!


latest_c_p_stats_tex = latest_c_p_stats.copy()
latest_c_p_stats_tex.drop(columns='current_total_employment', inplace=True)
latest_c_p_stats_tex['date_fmt'] = latest_c_p_stats_tex['date_fmt'].map(lambda date_: date_.strftime('%b %Y'))

latest_c_p_stats_tex['first_occurrence_employment'] = latest_c_p_stats_tex['first_occurrence_employment'].map(lambda x: "{:,.0f}".format(x))
latest_c_p_stats_tex['current_employment'] = latest_c_p_stats_tex['current_employment'].map(lambda x: "{:,.0f}".format(x))

latest_c_p_stats_tex.rename(columns={
    'date_fmt': 'Dátum',
    'providers': 'Poskytovateľ',
    'first_occurrence': 'D. nasadenia',
    'first_occurrence_employment': 'Obsluha (nasadenie)',
    'current_employment': 'Obsluha (aktuálne)',
    'current_share': r'\% z celku za mesiac',
}, inplace=True)

latest_c_p_stats_tex.to_latex("tables/httparchive_nel_latest_collector_provider_stats_table.tex", float_format=lambda x: '{:,.3f}'.format(x), index=False)
latest_c_p_stats_tex

#### GRAPH - Stats for the latest collector providers

In [12]:
from result_utils import MONTHS_MAP

to_plot = all_occurrences[all_occurrences['providers'].isin(latest_c_p_stats['providers'])].copy()

to_plot['date_fmt'] = to_plot['date'].map(lambda date: pd.to_datetime(date))
to_plot = to_plot.sort_values(by=['date_fmt'])
to_plot.reset_index(inplace=True)
to_plot = to_plot.reindex(columns=['date_fmt', 'date', 'providers', 'total_occurrences']) 

to_plot['Rok'] = to_plot['date_fmt'].dt.year
to_plot['Mesiac'] = to_plot['date_fmt'].dt.strftime('%m')

to_plot.sort_values(by=['date_fmt'], inplace=True)
# to_plot

to_plot.drop(columns=['date_fmt', 'date'], inplace=True)
# to_plot

fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(11, 5))

num_providers = to_plot['providers'].unique().size
subplot_cols = 5
subplot_rows = 2

sns.set_style('whitegrid')

for idx, provider in enumerate(latest_c_p_stats['providers']):
    current_plot_data = to_plot[to_plot['providers'] == provider].copy()
    current_plot_data.drop(columns=['providers'], inplace=True)

    current_plot_data['total_occurrences'] = current_plot_data['total_occurrences'].astype('int64')    
    current_plot_data = current_plot_data.pivot(index='Mesiac', columns='Rok', values='total_occurrences')
    current_plot_data.index = current_plot_data.index.map(MONTHS_MAP).astype('category')
    current_plot_data.fillna(0, inplace=True)

    current_ax = axes[idx // subplot_cols, idx % subplot_cols]
    sns.heatmap(current_plot_data, ax=current_ax, cbar_kws={'format': ticker.EngFormatter()})
    
    current_ax.set_xlabel("")
    current_ax.set_ylabel("")

    current_ax.set_title(provider)
    
    current_ax.tick_params(axis='y', which="major", rotation=0)
    current_ax.tick_params(axis='x', which="major", rotation=90)

fig.tight_layout()
fig.savefig('graphs/httparchive_nel_latest_collector_provider_stats.pdf')

### NEL Collector Providers employed by N domains


In [13]:
from results.result_utils import get_first_or_0

employment_result = pd.DataFrame({}, index=[
    '1',
    '2',
    '3-10',
    '11-100',
    '101-1K',
    'More',
])

for date in dates_to_visualize:
    month_collector_data = result[result['date'] == date].copy()
    
    month_data_col = []
    
    # month_result['1']
    next_val = month_collector_data[month_collector_data['as_primary'] == 1].count()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['2'] 
    next_val = month_collector_data[month_collector_data['as_primary'] == 2].count()
    month_data_col.append(get_first_or_0(next_val))

    # month_result['3-10']
    next_val = month_collector_data[(month_collector_data['as_primary'] >= 3) & (month_collector_data['as_primary'] <= 10)].count()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['3-10']
    next_val = month_collector_data[(month_collector_data['as_primary'] >= 11) & (month_collector_data['as_primary'] <= 100)].count()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['3-10']
    next_val = month_collector_data[(month_collector_data['as_primary'] >= 101) & (month_collector_data['as_primary'] <= 1000)].count()
    month_data_col.append(get_first_or_0(next_val))
    
    # month_result['3-10']
    next_val = month_collector_data[month_collector_data['as_primary'] >= 1001].count()
    month_data_col.append(get_first_or_0(next_val))
    
    
    employment_result[date_to_text_format(date)] = month_data_col
    
employment_result

### Total number of collectors found during the analysed time period


In [14]:
len(result['providers'].unique())